<a id='top'></a>
# Imaging Mode Data Calibration: Part 3 - Observation Level Calibrations
---
**Author**: Bryan Hilbert (hilbert@stsci.edu)| **Latest Update**: no need to update source catalog units, 14 December 2022

<div class="alert alert-block alert-warning">
    <h3><u><b>Notebook Goals</b></u></h3>
    <ul>Working with the Stage 3 Calibration Pipeline, we will:</ul>    
<ul>
    <li>Look at the different ways to call the pipeline</li>
    <li>Examine exactly what each pipeline step does to the science data</li>    
</ul>
</div>

## Table of Contents
* [Introduction](#intro)
* [Pipeline Resources and Documentation](#resources)
   * [Installation](#installation)
   * [Reference Files](#reference_files)
* [Imports](#Imports_ID)
* [Convenience Functions](#convenience_functions)
* [Download Data](#download_data)
* [Association Files](#associations)
* [Methods for calling steps/pipelines](#calling_methods)
* [Parameter Reference Files](#parameter_reffiles)
* [calwebb_image3 - Ensemble calibrations](#image3) 
   * [Run the entire pipeline](#image3_at_once)
       * [Using the run() method](#run_method)
       * [Using the call() method](#call_method)
       * [Using the command line](#command_line)
   * [Run the individual pipeline steps](#image3_step_by_step)
       * [The `WCS Refinement` step](#tweakreg)
       * [The `Sky Matching` step](#skymatch)
       * [The `Outlier Detection` step](#outlier_detection)
       * [The `Resample` step](#resample)
       * [The `Source Catalog` step](#source_catalog)
           * [Using the run() command](#srccat_run)
           * [Using the call() command](#srccat_call)
           * [Using the command line](#srccat_command_line)

<a id='intro'></a>
## Introduction

This notebook covers part 3 of the imaging mode data calibration module. In this notebook we'll review Stage 3 of the JWST calibration pipeline for imaging data, also known as *calwebb\_image3*. 

The [Stage 3 pipeline](https://jwst-pipeline.readthedocs.io/en/stable/jwst/pipeline/calwebb_image3.html) takes one or more calibrated slope images (`*_cal.fits` files) and combines them into a final mosaic image. It then creates a source catalog from this mosaic. Several steps are performed in order to prepare the data for the mosaic creation. These steps largely mirror what is done by [DrizzlePac](https://www.stsci.edu/scientific-community/software/drizzlepac.html) software when working with HST data. 

First, using common sources found across the input images, the WCS of each image is refined. Background levels are then matched across the inputs. Spurious sources (e.g. cosmic rays that were not flagged in the `jump` step during Stage 1 processing) are removed by comparing each individual input image to a median image. The indivudal images are combined into a single mosaic image. A source catalog is created based on the mosaic image. And finally, the individual exposures are updated using the information from the preceding steps. New versions of the individual calibrated slope images (`*_cal.fits` files) are produced that contain matched backgrounds, flagged spurious sources, and improved WCS objects. Also, updated [resampled](https://jwst-pipeline.readthedocs.io/en/stable/jwst/resample/main.html) images (`_i2d.fits` files) are created which all contain the final undistorted sky projection that is present in the mosaic image.

There are three final outputs. The first is updated copies of the input files. These updated files contain a consistent WCS, such that they overlap correctly. The second output is a final mosaic image created by drizzling the input images onto a distortion-free grid. And the final output is a source catalog wth basic photometry, created from the final mosaic image.

To illustrate how the steps of the pipeline change the input data, we will download several sample files and run them through the pipeline, examining the results at several places along the way.

All JWST imaging mode data, regardless of instrument, are processed through the *calwebb\_image3* pipeline. The steps and the order in which they are performed is the same for all data. For the purposes of this notebook, we will continue with the processing of the NIRCam data used in the Stages 1 and 2 notebooks. We will also provide example MIRI files that can be used in a separate exercise.

<a id='resources'></a>
## Pipeline Resources and Documentation

There are several different places to find information on installing and running the pipeline. This notebook will give a shortened description of the steps pulled from the detailed pipeline information pages, but to find more in-depth instructions use the links below.

* [JWST Documentation (JDox) for the Stage 3 pipeline](https://jwst-docs.stsci.edu/jwst-science-calibration-pipeline-overview/stages-of-jwst-data-processing/calwebb_image3) including short a short summary of what each step does.

* [High-level description of all pipeline stages and steps](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/main.html)

* [`jwst` package documentation](https://jwst-pipeline.readthedocs.io/en/latest/jwst/introduction.html) including how to run the pipeline, input/output files, etc.

* [`jwst` package GitHub repository, with installation instructions](https://github.com/spacetelescope/jwst/blob/master/README.md)

* [**Help Desk**](https://stsci.service-now.com/jwst?id=sc_cat_item&sys_id=27a8af2fdbf2220033b55dd5ce9619cd&sysparm_category=e15706fc0a0a0aa7007fc21e1ab70c2f): **If you have any questions or problems regarding the pipeline, submit a ticket to the Help Desk**

<a id='installation'></a>
### Installation

<div class="alert alert-block alert-info">
    During the JWebbinar, we will be working in a pre-existing environment where the <b>jwst</b> package has already been installed, so you won't need to install it yourself.
</div>

<div class="alert alert-block alert-warning">
If you wish to run this notebook outside of this JWebbinar, you will have to first install the <b>jwst</b> package.<br>

For more detailed instructions on the various ways to install the package, see the [installation instructions](https://github.com/spacetelescope/jwst/blob/master/README.md) on GitHub.

The easiest way to install the pipeline is via `pip`. Below we show how to create a new conda environment, activate that environment, and then install the latest released version of the pipeline. You can name your environment anything you like. In the lines below, replace `<env_name>` with your chosen environment name.

>`conda create -n <env_name> python`<br>
>`conda activate <env_name>`<br>
>`pip install jwst`

If you wish to install the development version of the pipeline, which is more recent than (but not as well tested compared to) the latest released version:

>`conda create -n <env_name> python`<br>
>`conda activate <env_name>`<br>
>`pip install git+https://github.com/spacetelescope/jwst`
    
</div>

<a id='reference_files'></a>
### Reference Files

[Calibration reference files](https://jwst-docs.stsci.edu/jwst-science-calibration-pipeline-overview/jwst-data-calibration-reference-files) are a collection of FITS and ASDF files that are used to remove instrumental signatures and calibrate JWST data. For example, the dark current reference file contains a multiaccum ramp of dark current signal to be subtracted from the data during the dark current subtraction step. 

When running a pipeline or pipeline step, the pipeline will automatically look for any required reference files in a pre-defined local directory. If the required reference files are not present, they will automatically be downloaded from the Calibration Reference Data System (CRDS) at STScI.

<div class="alert alert-block alert-info">
    During the JWebbinar, our pre-existing existing environment is set up to correctly use and store calibration reference files, and you do not need to set the environment variables below.
</div>
    
<div class="alert alert-block alert-warning">
If you wish to run this notebook outside of this JWebbinar, you will have to specify a local directory in which to store reference files, along with the server to use to download the reference files from CRDS. To accomplish this, there are two environment variables that should be set prior to calling the pipeline. These are the CRDS_PATH and CRDS_SERVER_URL variables. In the example below, reference files will be downloaded to the "crds_cache" directory under the home directory.

>`$ export CRDS_PATH=$HOME/crds_cache`<br>
>`$ export CRDS_SERVER_URL=https://jwst-crds-pub.stsci.edu`<br>
OR:<br>
`os.environ["CRDS_PATH"] = "/user/myself/crds_cache"`<br>
`os.environ["CRDS_SERVER_URL"] = "https://jwst-crds-pub.stsci.edu"`<br>

The first time you run the pipeline, the [CRDS server](https://jwst-pipeline.readthedocs.io/en/latest/jwst/introduction.html#crds) should download all of the context and reference files that are needed for that pipeline run, and dump them into the CRDS_PATH directory. Subsequent executions of the pipeline will first look to see if it has what it needs in CRDS_PATH and anything it doesn't have will be downloaded from the STScI cache. 
</div>

<div class="alert alert-block alert-warning">NOTE: Users at STScI should automatically have access to the Calibration Reference Data System (CRDS) cache for running the pipeline, and can skip setting these environment variables.</div>

[Top of Notebook](#top)

<a id='Imports_ID'></a>
## Imports

Import packages necessary for this notebook

In [ ]:
# Module with functions to get information about objects:
from glob import glob
#Modify the path to a directory on your machine
import os
os.environ["CRDS_PATH"] = "/home/jovyan/crds_cache/" # set appropriate path
os.environ["CRDS_SERVER_URL"] = "https://jwst-crds.stsci.edu"
#os.environ["CRDS_CONTEXT"] = "jwst_0966.pmap" # set context if needed

import shutil

# Numpy library:
import numpy as np

# To read association file
import json

# To download data
import requests

# To examine parameter reference files
import asdf

# Astropy tools:
from astropy.io import ascii, fits
import astropy.units as u
from astropy.utils.data import download_file
from astropy.visualization import ImageNormalize, ManualInterval, LogStretch, LinearStretch

Set up matplotlib for plotting

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl

# Use this version for non-interactive plots (easier scrolling of the notebook)
%matplotlib inline

# Use this version (outside of Jupyter Lab) if you want interactive plots
# %matplotlib notebook

# These gymnastics are needed to make the sizes of the figures
# be the same in both the inline and notebook versions
%config InlineBackend.print_figure_kwargs = {'bbox_inches': None}

mpl.rcParams['savefig.dpi'] = 80
mpl.rcParams['figure.dpi'] = 80

Import JWST pipeline-related modules

In [ ]:
# The entire calwebb_image3 pipeline
from jwst.pipeline import calwebb_image3

# Individual steps that make up calwebb_image3
from jwst.tweakreg import TweakRegStep
from jwst.skymatch import SkyMatchStep
from jwst.outlier_detection import OutlierDetectionStep
from jwst.resample import ResampleStep
from jwst.source_catalog import SourceCatalogStep
from jwst import datamodels
from jwst.associations import asn_from_list
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base

Check which version of the pipeline we are running:

In [ ]:
import jwst
print(jwst.__version__)

<a id='convenience_functions'></a>
## Define convenience functions and parameters

Here we define some functions that we will use repeatedly throughout the notebook.

In [ ]:
# Files created in this notebook will be saved
# in a subdirectory of the base directory called `Stage3`
output_dir = './'

In [ ]:
def download_files(files, output_directory, force=False):
    """Given a tuple or list of tuples containing (URL, filename),
    download the given files into the current working directory.
    Downloading is done via astropy's download_file. A symbolic link
    is created in the specified output dirctory that points to the
    downloaded file.
    
    Parameters
    ----------
    files : tuple or list of tuples
        Each 2-tuple should contain (URL, filename), where
        URL is the URL from which to download the file, and
        filename will be the name of the symlink pointing to
        the downloaded file.
        
    output_directory : str
        Name of the directory in which to create the symbolic
        links to the downloaded files
        
    force : bool
        If True, the file will be downloaded regarless of whether
        it is already present or not.
                
    Returns
    -------
    filenames : list
        List of filenames corresponding to the symbolic links
        of the downloaded files
    """
    # In the case of a single input tuple, make it a
    # 1 element list, for consistency.
    filenames = []
    if isinstance(files, tuple):
        files = [files]
        
    for file in files:
        filenames.append(file[1])
        if force:
            print('Downloading {}...'.format(file[1]))
            demo_file = download_file(file[0], cache='update')
            # Make a symbolic link using a local name for convenience
            if not os.path.islink(os.path.join(output_directory, file[1])):
                os.symlink(demo_file, os.path.join(output_directory, file[1]))
        else:
            if not os.path.isfile(os.path.join(output_directory, file[1])):
                print('Downloading {}...'.format(file[1]))
                demo_file = download_file(file[0], cache='update')
                # Make a symbolic link using a local name for convenience
                os.symlink(demo_file, os.path.join(output_directory, file[1]))
            else:
                print('{} already exists, skipping download...'.format(file[1]))
                continue
    return filenames

In [ ]:
def find_bad_pix_types(dq_value):
    """Given an integer representation of a series of bad pixel flags,
    identify which types of bad pixels the flags indicate.
    
    Parameters
    ----------
    dq_value : uint16
        Value associated with a set of bad pixel flags
        
    Returns
    -------
    bad_nums : list
        List of integers representing the bad pixel types
        
    bad_types : list
        List of bad pixel type names corresponding to bad_nums
    """
    # Change integer into a byte array
    bitarr = np.binary_repr(dq_value)
    
    # Find the bad pixel type associated with each bit where
    # the flag is set
    bad_nums = []
    bad_types = []
    for i, elem in enumerate(bitarr[::-1]):
        if elem == str(1):
            badval = 2**i
            bad_nums.append(badval)
            key = next(key for key, value in datamodels.dqflags.pixel.items() if value == badval)
            bad_types.append(key)
    return bad_nums, bad_types

In [ ]:
def overlay_catalog(data_2d, catalog, flux_limit=0, vmin=0, vmax=10,
                    title=None, units='MJy/str'):
    """Function to generate a 2D image of the data, 
    with sources overlaid.
    
    data_2d : numpy.ndarray
        2D image to be displayed
        
    catalog : astropy.table.Table
        Table of sources
    
    flux_limit : float
        Minimum signal threshold to overplot sources from catalog.
        Sources below this limit will not be shown on the image.
        
    vmin : float
        Minimum signal value to use for scaling
        
    vmax : float
        Maximum signal value to use for scaling
        
    title : str
        String to use for the plot title
                
    units : str
        Units of the data. Used for the annotation in the
        color bar
    """
    norm = ImageNormalize(data_2d, interval=ManualInterval(vmin=vmin, vmax=vmax),
                              stretch=LogStretch())
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(1, 1, 1)
    im = ax.imshow(data_2d, origin='lower', norm=norm)
    
    for row in catalog:
        if row['aper_total_flux'].value > flux_limit:
            plt.plot(row['xcentroid'], row['ycentroid'], marker='o',
                     markersize='3', color='red')

    plt.xlabel('Pixel column')
    plt.ylabel('Pixel row')
    
    fig.colorbar(im, label=units)
    fig.tight_layout()
    plt.subplots_adjust(left=0.15)
    
    if title:
        plt.title(title)

In [ ]:
def show_image(data_2d, vmin, vmax, xpixel=None, ypixel=None, title=None,
               scale='log', units='MJy/str'):
    """Function to generate a 2D, log-scaled image of the data, 
    with an option to highlight a specific pixel.
    
    data_2d : numpy.ndarray
        2D image to be displayed
        
    vmin : float
        Minimum signal value to use for scaling
        
    vmax : float
        Maximum signal value to use for scaling
        
    xpixel : int
        X-coordinate of pixel to highlight
        
    ypixel : int
        Y-coordinate of pixel to highlight
        
    title : str
        String to use for the plot title
        
    scale : str
        Specify scaling of the image. Can be 'log' or 'linear'
        
    units : str
        Units of the data. Used for the annotation in the
        color bar
    """
    if scale == 'log':
        norm = ImageNormalize(data_2d, interval=ManualInterval(vmin=vmin, vmax=vmax),
                              stretch=LogStretch())
    elif scale == 'linear':
        norm = ImageNormalize(data_2d, interval=ManualInterval(vmin=vmin, vmax=vmax),
                              stretch=LinearStretch())
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(1, 1, 1)
    im = ax.imshow(data_2d, origin='lower', norm=norm)
    
    if xpixel and ypixel:
        plt.plot(xpixel, ypixel, marker='o', color='red', label='Selected Pixel')

    fig.colorbar(im, label=units)
    plt.xlabel('Pixel column')
    plt.ylabel('Pixel row')
    if title:
        plt.title(title)

[Top of Notebook](#top)

<a id='download_data'></a>
## Download Data

For this module, we will use calbrated rate files from a NIRCam imaging exposure that is stored in Box. Let's download these files, as well as an association file and some parameter reference files.

In [ ]:
nircam_info = [('https://stsci.box.com/shared/static/d34352eb84s3i93ww1x786zz86mw5ni0.fits',
                'jw02727002001_02105_00001_nrcblong_cal.fits'),
               ('https://stsci.box.com/shared/static/0pl2ws5oqgan9w7k4el1tgzh2np0qtp1.fits',
                'jw02727002001_02105_00002_nrcblong_cal.fits'),
               ('https://stsci.box.com/shared/static/f88fwu91eo3dsmckttflwm13ld6wrx0w.fits',
                'jw02727002001_02105_00003_nrcblong_cal.fits'),
               ('https://stsci.box.com/shared/static/0ll6grrxo29x2o1o7klx9uoi3efnehkp.json',
                'jw02727-o002_20220802t161539_image3_asn.json'),
               ('https://stsci.box.com/shared/static/ja0gkd8c0x8p8konhr84wkuhwnpkqf4s.asdf',
                'jwst_nircam_pars-tweakregstep_0006.asdf'),
               ('https://stsci.box.com/shared/static/yahdw55fotwrh7i6hhcxksj97qkf7j4r.asdf',
                'nircam_pars-sourcecatalogstep_f444w_clear.asdf')
              ]
nircam_files = download_files(nircam_info, output_dir, force=False)

[Top of Notebook](#top)

<a id='associations'></a>
## Association Files

The Stage 3 pipeline must be called using a json-formatted file called an ["association" file](https://jwst-pipeline.readthedocs.io/en/stable/jwst/associations/index.html). When retrieving your observations from MAST, you will be able to download the association files for your data along with the fits files containing the observations.

The association file presents your data files in organized groups. Let's open the level 3 association file for the data to be processed in this notebook and look at its contents.

In [ ]:
asn_file = os.path.join(output_dir, 'jw02727-o002_20220802t161539_image3_asn.json')

In [ ]:
# Open the association file and load into a json object
with open(asn_file) as f_obj:
    asn_data = json.load(f_obj)

In [ ]:
asn_data

Here we see that the association file begins with a few lines of data that give high-level information about the association. The most important entry here is the `asn_rule` field. Association files have different formats for the different stages of the pipeline. You should be sure that the `asn_rule` matches the pipeline that you will be running. In this case we'll be running the Stage 3 pipeline, and we see that the `asn_rule` mentions "Level3", which is what we want.

Beneath these lines, we see the `products` field. This field contains a list of dictionaries that specify the files that belong to this association, and the types of those files. When the Stage 3 pipeline is run on this association file, all files listed here will be run through the calibration steps.

<a id='diy_association'></a>
##### Creating your own association files

As we will see below, when calling the steps of the pipeline one at a time, you can provide each step with either the output object from the preceding step, or you can provide an association file. Generally the former is easier to do. 

Since each step outputs modified observation files, in order to provide an association file for each step, you'll have to make a new association file for each step that lists the new observation files. Here we will show an example of how to create a new association file for the Stage 3 pipeline or pipeline steps. Keep in mind that you may also simply make a copy of an existing association file and update the member filenames.

We'll use the [`asn_from_list()` function](https://jwst-pipeline.readthedocs.io/en/stable/api/jwst.associations.asn_from_list.asn_from_list.html#jwst.associations.asn_from_list.asn_from_list) to create the new association file designed to be used in the `skymatch` step using as input the outputs from the `tweakreg` step. As input, we need the list of member files, as well as the product name. The product name will be prepended onto the output filenames by the pipeline or step that uses this association file.

In [ ]:
tweak_files = ['level3_lw_asn_0_tweakregstep.fits',
               'level3_lw_asn_1_tweakregstep.fits',
               'level3_lw_asn_2_tweakregstep.fits']
tweak_product = 'manual_asn_file'

In [ ]:
tweakreg_asn = asn_from_list.asn_from_list(tweak_files, rule=DMS_Level3_Base, product_name=tweak_product)

Here is our new association, containing the three files created by the `tweakreg` step.

In [ ]:
tweakreg_asn

Now save the new association to a json file.

In [ ]:
output_test = 'manual_tweakreg_asn.json'
with open(output_test, 'w') as outfile:
    name, serialized = tweakreg_asn.dump(format='json')
    outfile.write(serialized)

Another way to create an association file is to use the [asn_from_list command line tool](https://jwst-pipeline.readthedocs.io/en/stable/api/jwst.associations.asn_from_list.asn_from_list.html#jwst.associations.asn_from_list.asn_from_list). The same association file that we created above can be created via the following command on the command line. **NOTE:** The command below will not work correctly until we have run the Tweakreg step and created some `*_tweakregstep.fits` files.

<div class="alert alert-block alert-info">
    
```
asn_from_list -o manual_tweakreg_asn.json --product-name manual_asn_file level3_lw_asn_?_tweakregstep.fits
```  
</div>

You can see how creating a new association file for each step of the pipeline would become cumbersome. For the step-by-step calls to the pipeline below, we will input the preceding step's output object into each step.

[Top of Notebook](#top)

<a id='calling_methods'></a>
## Methods for calling steps/pipelines

There are three common methods by which the pipeline or pipeline steps can be called. From within python, the `run()` and `call()` methods of the pipeline or step classes can be used. Alternatively, the `strun` command can be used from the command line. Within this notebook, in the section where we [call the entire pipeline](#image3_at_once), as well as the section where we [call the WCS Refinement](#tweakreg_call) step, we show examples of all three methods. For the remainder of the pipeline steps, we will focus on using the `run()` method.

When using the `call()` method or `strun`, optional input parameters can be specified via [parameter reference files](#parameter_reffiles). When using the `run()` method, these parameters are instead specified within python.

<a id='parameter_reffiles'></a>
## Parameter Reference Files

When calling a pipeline or pipeline step using the `call()` method or the command line, [parameter reference files](https://jwst-pipeline.readthedocs.io/en/stable/jwst/stpipe/config_asdf.html#config-asdf-files) can be used to specify values for input parameters. These reference files are [asdf](https://asdf.readthedocs.io/en/stable/) format and appear somewhat similar to json files when examined in a text editor. 

Currently, CRDS contains a default version of a parameter reference file for the [WCS refinement](#tweakreg) step only. When using the `call()` method, if you do not specify a parameter reference file name in the call, the pipeline or step will retrieve and use the appropriate file from CRDS, which will then run the pipeline or step with the parameter values in that file. If you provide the name of a parameter reference file, then the parameter values in that file will take precedence. For any parameter not specified in your parameter reference file, the pipeline will use the default value.

When using `strun`, the parameter reference file is a required input.

Let's take a look at the contents of a parameter reference file. NIRCam does not currently have a parameter reference file for the Stage 3 imaging pipeline, so we'll look at the parameter reference file for the WCS refinement step. We'll open it using the asdf package, and use the `tree` attribute to see what's inside:

In [ ]:
tweak_param_reffile = 'jwst_nircam_pars-tweakregstep_0006.asdf'

In [ ]:
tweak_params = asdf.open(tweak_param_reffile)

In [ ]:
tweak_params.tree

The top part of the file contains various metadata entries about the file itself. Below that, you'll see a `'name'` entry, which lists `Tweakreg` as the class to which these parameters apply. The next line contains the `parameters` entry, which lists parameters and values attached to the step. If you look through these entries, you'll see the same parameters and values that we specified manually when using the `run()` method above.

In [ ]:
# Don't forget to close the file
tweak_params.close()

[Top of Notebook](#top)

---
<a id='image3'></a>
## The calwebb_image3 pipeline: Ensemble processing

In the sections below, we will run the Stage 3 pipeline using an association file containing several NIRCam exposures. We will first call the entire *calwebb_image3* pipeline itself. The pipeline is a wrapper which will string together all of the appropriate steps in the proper order.

After running the entire pipeline, we will go back to the original calibrated slope images and manually run them through each of the steps that comprise the Stage 3 pipeline. For each step we will describe in more detail what is going on and examine how the exposure files have changed.

See [Figure 1](https://jwst-docs.stsci.edu/jwst-science-calibration-pipeline-overview/stages-of-jwst-data-processing/calwebb_image3) on the calwebb_image3 algorithm page for a map of the steps are performed on the input data.

<a id='image3_at_once'></a>
### Run the entire `calwebb_image3` pipeline

In this section we show how to run the entire calwebb_image3 pipeline with a single call. 

We show all three methods for calling the pipeline.


We set parameter values for some of the individual steps, save some outputs, etc, and then call the pipeline.


<a id='run_method'></a>
#### Call the pipeline using the run() method

When using the `run()` method to execute a pipeline (or step), the pipeline class is first instantiated without the data to be processed. Optional input parameters are specified using attributes of the class instance. Finally, the call to the `run()` method is made and the data are supplied.  See here for [more examples of the run() method](https://jwst-pipeline.readthedocs.io/en/stable/jwst/stpipe/call_via_run.html).

The `run()` method does not take any kind of parameter reference file as input. If you wish to set values for various parameters, you must do that manually. Below, we set several paramaters in order to show how it's done. 

How do you know what parameters are available to be set and what their default values are? The `spec` property for individual steps will list them. The property is less useful for the pipelines themselves, as it does not show the parameters for the steps compirising the pipeline.

All steps and pipelines have several common parameters that can be set. 

* `save_results` specifies whether or not to save the output of that step/pipeline to a file. The default is False.
* `output_dir` is the directory into which the output files will be saved.
* `output_file` is the base filename to use for the saved result. Note that each step/pipeline will add a custom suffix onto output_file. 

Let's look at the available parameters for the tweakreg step and the source catalog step, and manually set some of these in our call to `run()`.

In [ ]:
print(TweakRegStep.spec)

In [ ]:
print(SourceCatalogStep.spec)

<a id='detector1_using_run'></a>

<div class="alert alert-block alert-info">
Finally, let's run the pipeline. The output can be a little overwhelming. There will be multiple log entries printed to the screen for each step.
</div>

In [ ]:
# Create an instance of the pipeline class
image3 = calwebb_image3.Image3Pipeline()

# Set some parameters that pertain to the
# entire pipeline
image3.output_dir = output_dir
image3.save_results = True

# Set some parameters that pertain to some of
# the individual steps
image3.tweakreg.snr_threshold = 10.0  # 5.0 is the default
image3.tweakreg.kernel_fwhm = 2.302  # 2.5 is the default
image3.tweakreg.brightest = 200  # 100 is the default
image3.tweakreg.fitgeometry = 'shift'
image3.tweakreg.abs_refcat = 'GAIADR2'
image3.tweakreg.searchrad = 3.0

image3.source_catalog.kernel_fwhm = 2.302  # pixels
image3.source_catalog.snr_threshold = 10.

# Call the run() method
image3.run(asn_file)

This cell will take a few minutes to run. While we're waiting, let's look at the other two methods that can be used to call the pipeline. Then we'll come back here and look at the log meessages output by this cell so we can see what happened.

<a id='call_method'></a>
#### Call the pipelne using the call() method

When using the `call()` method, a single command will instantiate and run the pipeline (or step). The input data and optional parameter reference files are supplied in this single command. In this case, any desired input parameters cannot be set after instantiation, as with the `run()` method. See here for [example usage of call() method](https://jwst-pipeline.readthedocs.io/en/stable/jwst/stpipe/call_via_call.html).

The commands below will call the pipeline using the `call()` method. In this case, there is no default parameter reference file in CRDS. Therefore, when using the `call()` method, we won't specify a paramter reference file, and the pipeline will fall back to using default parameter values that are defined in the code.

Since we just ran the pipeline with the `run()` method above, we won't actually execute the call to `call()`. But if you wish to try it out, use the pull-down menu above to change the cell to be 'Code', and then execute it. (Or, Click 'Cell' > 'Cell Type' > 'Code')

<div class="alert alert-block alert-info">

<b>Method #1:</b>
Provide the name of the association file, and set pipeline-specific input paramters. Here, we don't set any step-specific paramters, so the pipeline will fall back to using default values for everything.
</div>

<div class="alert alert-block alert-info">

<b>Method #2:</b>
In this case, build a nested dictionary that specifies parameter values for various steps, and provide it in the call to call().
</div>

<a id='command_line'></a>
#### Call the pipeline from the command line

Calling a pipeline or step from the command line is similar to using the `call()` method. The data file to be processed, along with an optional parameter reference file and optional parameter/value pairs can be provided to the `strun` command. See here for [additional examples of command line calls](https://jwst-pipeline.readthedocs.io/en/stable/jwst/introduction.html?highlight=%22command%20line%22#running-from-the-command-line).

Since NIRCam does not have a parameter reference file for calwebb_image3, we show how to use the `strun` command and manually update parameter values. 

<div class="alert alert-block alert-info">

<b>Method #1:</b>
We provide the name of the pipeline class, the observation file, and explicitly set pipeline- and step-specific parameters. You can see that the command quickly becomes quite large with the added parameter settings. 
    
```
    strun jwst.pipeline.Image3Pipeline level3_lw_asn.json --save_results=True --output_dir='./' --steps.tweakreg.kernel_fwhm=2.302 --steps.tweakreg.snr_threshold=10. --steps.tweakreg.brightest=20 --steps.source_catalog.kernel_fwhm=2.302 --steps.source_catalog.snr_threshold=10.
```
</div>

### Examine the outputs

Get the input filenames from the association file

In [ ]:
input_files = [item['expname'] for item in asn_data['products'][0]['members']]       

In [ ]:
input_files

Define the names of the other output files.

In [ ]:
mosaic_file = os.path.join(output_dir, 'jw02727-o002_t062_nircam_clear-f444w_i2d.fits')
source_cat_file = os.path.join(output_dir, 'jw02727-o002_t062_nircam_clear-f444w_cat.ecsv')
segmentation_map_file = os.path.join(output_dir, 'jw02727-o002_t062_nircam_clear-f444w_segm.fits')
cr_flagged_files = [item.replace('cal.fits', 'crf.fits') for item in input_files]

Read in the final mosaic image and display

In [ ]:
mosaic = datamodels.open(mosaic_file)

In [ ]:
show_image(mosaic.data, vmin=0.32, vmax=5)

Let's look at the segmentation map that was created by the `source_catalog` step. This shows which pixels are associated with the identified sources.

In [ ]:
seg_map = fits.getdata(segmentation_map_file)

In [ ]:
show_image(seg_map, vmin=0, vmax=5, scale='linear')

And now examine the actual source catalog. For each source, the catalog lists the location, along with flux and AB/Vega magnitude values in three different apertures, as well as calculated values for an infinite aperture. Within the documentation, you can see the [full list of column definitions](https://jwst-pipeline.readthedocs.io/en/stable/jwst/source_catalog/main.html#source-catalog-table).

In [ ]:
source_cat = ascii.read(source_cat_file)

In [ ]:
source_cat

Finally, let's overlay the source catalog on top of the mosaic image. In order to cut down on the number of spurious detections, we only show sources above a minimum flux limit. Another way to cut down on the number of spurious detections would be to change some of the `source_catalog` parameter values when calling the pipeline above.

In [ ]:
overlay_catalog(mosaic.data, source_cat, flux_limit=5e-7, vmin=0.3, vmax=10,
                title='Final mosaic with source catalog')

[Top of Notebook](#top)

<a id='image3_step_by_step'></a>
## Run the individual pipeline steps

In the sections below we run the steps contained within calwebb_image3 one at a time, in order to more clearly see what each step is doing.

<a id='tweakreg'></a>
### The `WCS refinement` step

#### Summary

This step, called the `tweakreg` step, mimics the behavior of the tweakreg step of Astrodrizzle. Given a series of images, it identifies point sources that are common to two or more images, and uses those sources' locations to correct the WCS of the input images. The tweaks are such that when the images are later combined into a final mosiac image, the WCS of the input images will align on the sky. 

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/tweakreg/README.html) of the step.

#### Arguments

There are many [optional input arguments](https://jwst-pipeline.readthedocs.io/en/stable/jwst/tweakreg/README.html#step-arguments).

#### Reference files used

This step does not use any reference files.

#### Parameter reference files

There are filter-dependent parameter reference files in CRDS for this step. 

#### Run the step

##### Using the `run()` method

The tweakreg step takes as input the same association file that we used as input to the entire pipeline above.

With the run() method specifically, we need to set any non-default parameter values manually. In this case, the entries below mimic those in the parameter reference file that we will use with the `call()` method next.

Let's look at what parameters are available to be set.

In [ ]:
print(TweakRegStep.spec)

In [ ]:
# Create instance, set parameters, and run
tweakreg = TweakRegStep()
tweakreg.kernel_fwhm = 2.302   # Gaussian FWHM in pixels
tweakreg.snr_threshold = 10.0  # SNR threshold above background
tweakreg.brightest = 200       # Number of brightest objects to keep
tweakreg.fitgeometry = 'shift'
tweakreg.abs_refcat = 'GAIADR2'
tweakreg.searchrad = 3.0

tweakreg.save_results = True
tweak = tweakreg.run(asn_file)

The step saves the results in new fits files with updated WCS information. Let's look at the difference in the WCS before and after the step by loading the WCS objects, and calculating the RA, Dec at one pixel. The name of the output files from this step can be found at the bottom of the log above. Another way to get the filename is by looking in the "filename" metadata entry for one of the entries in the ModelContainer output by the tweakreg step, as shown below. 

In [ ]:
cal_file = 'jw02727002001_02105_00001_nrcblong_cal.fits'
tweak_file = tweak[0].meta.filename

In [ ]:
# Open the files using datamodels
cal_data = datamodels.open(cal_file)
tweak_data = datamodels.open(tweak_file)

In [ ]:
cal_detector_to_world = cal_data.meta.wcs.get_transform('detector', 'world')
run_detector_to_world = tweak_data.meta.wcs.get_transform('detector', 'world')

Let's look at RA, Dec in the center of the detector.

In [ ]:
x, y = (1024, 1024)

The reported RA, Dec at that location is different in the data before and after the Tweakreg step has been run.

In [ ]:
print(cal_detector_to_world(x, y))
print(run_detector_to_world(x, y))

In [ ]:
# What is the shift in the WCS before/after this step?
delta_ra = run_detector_to_world(x, y)[0] - cal_detector_to_world(x, y)[0]
delta_dec = run_detector_to_world(x, y)[1] - cal_detector_to_world(x, y)[1]
print('Shift in RA, Dec is ({:.4f}, {:.4f}) arcsec'.format(delta_ra * 3600., delta_dec * 3600.))
print('This is ({:.3f}, {:.3f}) pixels.'.format(delta_ra * 3600. / .062, delta_dec * 3600. / 0.062))

Just for fun, let's look at the contents of the WCS object in one of the files saved by the tweakreg step.

In [ ]:
tweak_data.meta.wcs

[Top of Notebook](#top)

<a id='skymatch'></a>
## The `Sky matching` step

#### Summary

This step calculates sky values in overlapping regions of the input images. Sky values can be computed for each image separately or in a way that matches the sky levels amongst the collection of images so as to minimize their differences.

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/skymatch/README.html) of the step. Note that there are several possible methods for calculating the sky values. 

#### Arguments

There are [several optional arguments](https://jwst-pipeline.readthedocs.io/en/stable/jwst/skymatch/README.html#step-arguments) for this step.

#### Reference files used

This step does not use any reference files.

#### Parameter reference files
There are currently no parameter reference files for this step

#### Run the step

The easiest way to call this step is to supply the output object from the tweakreg call above. The other option would be to supply an association file. However, since the tweakreg step saved modified files, we would have to create a new association file that contains these new files, and then supply that association file in the call to skymatch. This pattern is the same for all of the steps in calwebb_image3, since all steps now expect association files for inputs, rather than individual files. 

To create a new association file, follow the steps outlined in the [Creating your own association files](#diy_association) section above. Keep in mind that you could also simply make a copy of an existing association file and update the member filenames.

Let's also see what parameters are available to be set.

In [ ]:
print(SkyMatchStep.spec)

In [ ]:
skymatch = SkyMatchStep()
skymatch.skymethod = 'global+match' # this is the default. Set here as an example
skymatch.save_results = True
skymatch.subtract = False
sky = skymatch.run(tweak)

As with the tweakreg step above, we can find the skymatch output filenames by looking at the bottom few entries of the log. Again, a programmatic way to get the filenames is to look in the metadata of the output ModelContainer.

In [ ]:
# Get the three filenames of the output files
for i in range(3):
    print(sky[i].meta.filename)

If you request to save the output from this step, two new header keywords are added to the primary header of the outputs. In each file, the `BKGLEVEL` keyword lists the computed background level, and the `BKGSUB` keyword says whether or not the background has been subtracted from the data.

In this case, we kept the default behavior of not subtracting the background from the data, as shown below.

In [ ]:
# sky_file = 'step_SkyMatchStep_2_skymatchstep.fits'
# or
sky_file = sky[0].meta.filename

In [ ]:
sky_header = fits.getheader(sky_file, 0)
print('Computed background level: {}'.format(sky_header['BKGLEVEL']))
print('Background subtracted: {}'.format(sky_header['BKGSUB']))

Look at the data values for one pixel before and after the SkyMatch step, in order to see that the background has indeed not been subtracted.

In [ ]:
tweak_data.data[2000, 250], sky[0].data[2000, 250]

[Top of Notebook](#top)

<a id='outlier_detection'></a>
## The `Outlier Detection` step

#### Summary

This step uses the collection of input files to identify and flag any cosmic rays or other transient image artifacts that were not flagged by the jump step in calwebb_detector1. While the jump step looked for large pixel-based deviations in the signal from group-to-group within an integration, the outlier detection step looks for large sky-based exposure-to-exposure devations in the signal. If a given location on the sky shows no signal above the noise in 4 out of 5 exposures, but a bright source in the remaining exposure, the outlier detction step will flag in the DQ map the pixels containing the source in the fifth exposure. These pixels will be ignored when the exposures are later combined into a final mosaic image.

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/outlier_detection/main.html) of the step.

#### Arguments

There are [numerous optional arguments](https://jwst-pipeline.readthedocs.io/en/stable/jwst/outlier_detection/arguments.html) for this step, including several that apply to the resample step, which this step makes use of.


#### Reference files used

This step does not use any reference files.

#### Run the step

List the available parameters:

In [ ]:
print(OutlierDetectionStep.spec)

In [ ]:
# Run the step
outlier_detection = OutlierDetectionStep()
outlier_detection.save_results = True
outlier = outlier_detection.run(sky)

In [ ]:
outlier_file = outlier[0].meta.filename

In [ ]:
# Open using datamodels
outlier_data = datamodels.open(outlier_file)

In [ ]:
# Get a list of pixels where DQ flags were changed
new_flags = np.where(cal_data.dq != outlier_data.dq)

In [ ]:
print("Found {} pixels with updated DQ flag values.".format(len(new_flags[0])))

Let's have a look at the DQ value for one of these pixels before and after the outlier detection step has run, in order to see what has changed.

In [ ]:
index = 8121
y = new_flags[0][index]
x = new_flags[1][index]
print('Pixel x, y = ({}, {})'.format(x, y))
print('Before: {}'.format(cal_data.dq[y, x]))
print('After: {}'.format(outlier_data.dq[y, x]))

Below we see that the 'OUTLIER' and the 'DO_NOT_USE' flags are new. 

In [ ]:
find_bad_pix_types(cal_data.dq[y, x])

In [ ]:
find_bad_pix_types(outlier_data.dq[y, x])

The image data itself remains unchanged.

In [ ]:
print(outlier_data.data[y, x], sky[0].data[y, x])

When the Resample step is run next in order to create the final mosaic image, all pixels flagged as DO_NOT_USE will be ignored.

[Top of Notebook](#top)

<a id='resample'> </a>
## The `Resample` step

#### Summary

We initially saw this step in calwebb_image2, where it was used to resample individual images onto a distortion-free pixel grid. This time, the Resample step works on the set of input images, which now all have a consistent WCS, thanks to the tweakreg step. The input images are combined into a final mosaic as they are resampled onto a distortion-free grid. The output of this step is the final image output of the Stage 3 pipeline. 

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/resample/main.html) of the step.

#### Arguments

There is a list of [optional Astrodrizzle-style](https://jwst-pipeline.readthedocs.io/en/stable/jwst/resample/arguments.html) input parameters that can be used to customize the resampling process.

#### Reference files used

This step uses the [`DRIZPARS`](https://jwst-pipeline.readthedocs.io/en/stable/jwst/resample/reference_files.html) reference file. This file contains Astrodrizzle-style keywords that can be used to control the details of the resampling.

#### Run the step

List the available parameters and their default values:

In [ ]:
print(ResampleStep.spec)

In [ ]:
# Run the step
resample = ResampleStep()
resample.save_results = True
resamp = resample.run(outlier)

In [ ]:
#resamp_file = 'step_ResampleStep_resamplestep.fits'
# or
resamp_file = resamp.meta.filename

In [ ]:
show_image(resamp.data, 0., 10)

In [ ]:
resamp.data.shape

[Top of Notebook](#top)

<a id='source_catalog'> </a>
## The `Source Catalog` step

#### Summary

This step creates a catalog of source photometry and morphology information. Sources are identified using [Photutils' image segmentation](https://photutils.readthedocs.io/en/latest/segmentation.html) method. The output is an ASCII file containing a table of source locations and aperture photometry results.

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/source_catalog/main.html) of the step.

#### Arguments

There is a list of [optional input parameters](https://jwst-pipeline.readthedocs.io/en/stable/jwst/source_catalog/arguments.html) that can be used to customize the resampling process.

#### Reference files used

This step uses the [`APCORR` and `ABVEGAOFFSET`](https://jwst-pipeline.readthedocs.io/en/stable/jwst/source_catalog/reference_files.html) reference files. The `APCORR` reference file contains the factors necessary to correct aperture photometry results to the equivalent of an infinite aperure. The `ABVEGAOFFSET` reference file contains data necessary for converting from AB to Vega magnitudes.

<a id='srccat_run'></a>
#### Run the step with the run() method

List the available parameters and their default values:

In [ ]:
print(SourceCatalogStep.spec)

In [ ]:
# Run the step after setting some parameters
srccat = SourceCatalogStep()
srccat.save_results = True
srccat.kernel_fwhm = 2.302  # pixels
srccat.snr_threshold = 50.

# Note that if you call source catalog and input the datamodel
# instance from the resample step, the source catalog step will
# change the units of the data from MJy/str to Jy.
source_cat = srccat.run(resamp)

<a id='srccat_call'></a>
#### Run the step with the call() method

Here we will call the source catalog step using the call method and supplying a parameter reference file. 

Let's have a look at the contents of the parameter reference file. About halfway down the `parameters` field, which contains a dictionary of parameter names and values. By editing these values, you can change the parameter values used by the source catalog step. Note that in this case, the file has been modified to have the same parameter values as were used in the run() method call above.

In [ ]:
src_cat_param_reffile = 'nircam_pars-sourcecatalogstep_f444w_clear.asdf'
src_cat_params = asdf.open(src_cat_param_reffile)
src_cat_params.tree

In [ ]:
src_cat_params.close()

If you were to use the `call()` method below but not specify a parameter reference file, the pipeline would query CRDS and run using the appropriate default parameter reference file for the filter/pupil values of your observation. Only if you wish to change from the default parameters do you need to specify a parameter reference file in your `call()` statement. If you wish to run the `call()` command in the cell below, change the type to 'Code' in the pull down menu towards the top of the window.

<a id='srccat_command_line'></a>
#### Run the step with the command line

Here we show the commands that can be used from the command line to run the source catalog step in the same ways as above. Note that in this case, we'll need to use a new association file that lists all of the files output by the preceding Resample step.

<div class="alert alert-block alert-info">
    
In this example, all parameter values are set in the parameter reference file.
    
```
    strun nircam_pars-sourcecatalogstep_f444w_clear.asdf step_ResampleStep_resamplestep.fits
```
    
In this example, no parameter reference file is provided, and non-default parameter values are set manually.   
    
```
    strun jwst.source_catalog.SourceCatalogStep step_ResampleStep_resamplestep.fits --save_results=True --output_dir='./' --kernel_fwhm=2.302 --save_results=True --snr_threshold=10 
```


</div>

#### Examine the results

In [ ]:
source_cat

Overlay the catalog sources on the data

In [ ]:
overlay_catalog(resamp.data, source_cat, flux_limit=0, vmin=0, vmax=10,
                title='Final mosaic with source catalog')

This is the end of the Stage 3 pipeline for imaging. The outputs from this pipeline are the final calibrated products and are ready for post-pipeline analysis.

[Top of Notebook](#top)